In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow_decision_forests as tfdf
from IPython.display import display
print("Found TensorFlow Decision Forests v" + tfdf.__version__)

Found TensorFlow Decision Forests v1.0.1


In [2]:
# credit_data = pd.read_csv('german_credit_data.csv')
# credit_data.to_hdf('credit_data.h5', key='df', mode='w')
credit_data = pd.read_hdf('credit_data.h5', 'df')  
display(credit_data)

,CustomerID,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,713a336c-a255-4e2d-9d57-90b3e99e2f06,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,140b363f-a3fe-4828-a33f-7284dfdb3969,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,43b7b51d-5eda-4860-b461-ebef3d3436f4,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,f40eaf08-e6d1-4765-ab20-c5f7faca1635,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,1728910a-d3ff-4799-ac50-203a3a58a3fb,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,...,unknown,57,none,own,2,skilled,1,none,yes,Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,e77fa77b-78da-4607-a2fa-ede36c1e968f,greater_200,27,credits_paid_to_date,furniture,4650,less_100,1_to_4,3,male,...,savings_insurance,40,none,own,1,skilled,1,none,yes,No Risk
4996,6e71db4b-375c-42e1-b4a8-3292c007967a,0_to_200,11,prior_payments_delayed,furniture,250,greater_1000,4_to_7,3,male,...,car_other,32,bank,own,1,unemployed,1,none,yes,No Risk
4997,db501d22-e0c9-4f38-bf05-7f2c1df35395,no_checking,32,outstanding_credit,appliances,6536,unknown,greater_7,5,male,...,unknown,54,stores,own,2,unskilled,2,yes,yes,Risk
4998,802055d6-6aa7-4532-bf1c-0b8b114b483d,0_to_200,38,outstanding_credit,other,1597,500_to_1000,greater_7,3,female,...,savings_insurance,27,stores,own,1,skilled,1,none,yes,Risk


In [3]:
data = credit_data.copy()

In [4]:
# Name of the label column.
label = 'Risk'
id_column = 'CustomerID'
data[label].value_counts().to_frame().T

,No Risk,Risk
Risk,3330,1670


In [5]:
label_new = label + '_target'
data[label_new] = data[label].map({'No Risk':0, 'Risk':1})
data = data.drop(columns=[label])
data[label_new].value_counts().to_frame().T

,0,1
Risk_target,3330,1670


In [6]:
print(data.dtypes)

CustomerID                  object
CheckingStatus              object
LoanDuration                 int64
CreditHistory               object
LoanPurpose                 object
LoanAmount                   int64
ExistingSavings             object
EmploymentDuration          object
InstallmentPercent           int64
Sex                         object
OthersOnLoan                object
CurrentResidenceDuration     int64
OwnsProperty                object
Age                          int64
InstallmentPlans            object
Housing                     object
ExistingCreditsCount         int64
Job                         object
Dependents                   int64
Telephone                   object
ForeignWorker               object
Risk_target                  int64
dtype: object


In [7]:
def remove_all_by_values(list_obj, values):
    list_obj_rem = list_obj.copy()
    for value in set(values):
        while value in list_obj_rem:
            list_obj_rem.remove(value)
    return list_obj_rem

In [8]:
label = label_new
id_column = 'CustomerID'
outColumns = [label, id_column]
# Get  columns whose data type is object i.e. string
filteredColumns = data.dtypes[data.dtypes == object]
# list of columns whose data type is object i.e. string
objectColumns = list(filteredColumns.index)
objectFeatures = remove_all_by_values(objectColumns, outColumns)

filteredColumns = data.dtypes[data.dtypes == np.int64]
objectColumns = list(filteredColumns.index)
numericFeatures = remove_all_by_values(objectColumns, outColumns)


# Target column name.
TARGET_COLUMN_NAME = label
# Numeric feature names.
NUMERIC_FEATURE_NAMES = numericFeatures
# Categorical features and their vocabulary lists.
CATEGORICAL_FEATURE_NAMES = objectFeatures

print(TARGET_COLUMN_NAME)
print(NUMERIC_FEATURE_NAMES)
print(CATEGORICAL_FEATURE_NAMES)

Risk_target
['LoanDuration', 'LoanAmount', 'InstallmentPercent', 'CurrentResidenceDuration', 'Age', 'ExistingCreditsCount', 'Dependents']
['CheckingStatus', 'CreditHistory', 'LoanPurpose', 'ExistingSavings', 'EmploymentDuration', 'Sex', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans', 'Housing', 'Job', 'Telephone', 'ForeignWorker']


In [9]:
train = 0.7
test = 1 - train
label_lst = [label]
train_data, test_data = train_test_split(data, test_size=test, random_state=0, stratify=data[label_lst])
print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
train_data[label].value_counts().to_frame().T

Train data shape: (3499, 22)
Test data shape: (1501, 22)


,0,1
Risk_target,2330,1169


In [10]:
def specify_feature_usages():
    feature_usages = list()

    for feature_name in NUMERIC_FEATURE_NAMES:
        feature_usage = tfdf.keras.FeatureUsage(
            name=feature_name, semantic=tfdf.keras.FeatureSemantic.NUMERICAL
        )
        feature_usages.append(feature_usage)

    for feature_name in CATEGORICAL_FEATURE_NAMES:
        feature_usage = tfdf.keras.FeatureUsage(
            name=feature_name, semantic=tfdf.keras.FeatureSemantic.CATEGORICAL
        )
        feature_usages.append(feature_usage)

    return feature_usages

In [11]:
# Convert the dataset into a TensorFlow dataset.
batch_size = int(train_data.shape[0]/10)
train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(train_data, label=TARGET_COLUMN_NAME, batch_size=batch_size)
test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(test_data, label=TARGET_COLUMN_NAME, batch_size=batch_size)

/opt/conda/lib/python3.10/site-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  features_dataframe = dataframe.drop(label, 1)
/opt/conda/lib/python3.10/site-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  features_dataframe = dataframe.drop(label, 1)


In [12]:
# Minimum number of examples in a node.
MIN_EXAMPLES = int(train_data.shape[0]*0.05)
# Maximum depth of the tree. max_depth=1 means that all trees will be roots.
MAX_DEPTH = 12
# Ratio of the training dataset used to monitor the training. Require to be >0 if early stopping is enabled.
VALIDATION_RATIO = 0.2

# Train the model
model = tfdf.keras.CartModel(features=specify_feature_usages(),
        exclude_non_specified_features=True,
        # max_depth=MAX_DEPTH,
        # min_examples=MIN_EXAMPLES,
        # validation_ratio=VALIDATION_RATIO,
        task=tfdf.keras.Task.CLASSIFICATION)

model.compile(metrics=[keras.metrics.PrecisionAtRecall(0.5)])

Use /tmp/tmpfe6aptwz as temporary training directory


In [13]:
model.fit(train_dataset, batch_size=batch_size)

Reading training dataset...
Training dataset read in 0:00:02.540426. Found 3499 examples.
Training model...
Model trained in 0:00:00.041348
Compiling model...


[INFO kernel.cc:1176] Loading model from path /tmp/tmpfe6aptwz/model/ with prefix da8d26d205904198
[INFO abstract_model.cc:1248] Engine "RandomForestOptPred" built
[INFO kernel.cc:1022] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [14]:
with open("model.html", "w") as html_file:
    html_file.write(tfdf.model_plotter.plot_model(model, max_depth=MAX_DEPTH))

In [15]:
#print(model.summary())

# The input features
model.make_inspector().features()

# The feature importances
print(f"Available variable importances:")
for importance in model.make_inspector().variable_importances().keys():
    print("\t", importance)

model.make_inspector().variable_importances()["SUM_SCORE"]

Available variable importances:
	 NUM_AS_ROOT
	 SUM_SCORE
	 NUM_NODES
	 MEAN_MIN_DEPTH


[("EmploymentDuration" (4; #5), 306.952433668077),
 ("CheckingStatus" (4; #1), 104.42077428288758),
 ("LoanDuration" (1; #14), 62.80766389518976),
 ("Age" (1; #0), 40.701236886903644),
 ("CreditHistory" (4; #2), 31.263962658122182),
 ("Sex" (4; #18), 31.05180247500539),
 ("Housing" (4; #9), 29.11526518687606),
 ("OthersOnLoan" (4; #16), 25.114207178354263),
 ("CurrentResidenceDuration" (1; #3), 23.401690114289522),
 ("LoanPurpose" (4; #15), 21.98563720472157),
 ("ExistingCreditsCount" (1; #6), 18.241982291452587),
 ("LoanAmount" (1; #13), 15.910028874874115),
 ("OwnsProperty" (4; #17), 9.100025609135628),
 ("ExistingSavings" (4; #7), 8.510946996510029),
 ("Telephone" (4; #19), 5.046664588153362),
 ("Job" (4; #12), 4.766048341989517)]

In [16]:
print("Evaluate")
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

Evaluate
5/5 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - precision_at_recall: 0.7654


{'loss': 0.0, 'precision_at_recall': 0.7653631567955017}

In [17]:
predictions = model.predict(test_dataset)
y_true = test_data[label]
roc_auc = roc_auc_score(y_true, predictions)
print(f"The ROC AUC score is {roc_auc}")

5/5 [==============================] - 0s 2ms/step
The ROC AUC score is 0.8093902195608782


In [18]:
y_pred = np.where(model.predict(test_dataset) < 0.5, 0, 1).ravel()
y_true = test_data[label]
print(confusion_matrix(y_true, y_pred))
print(f1_score(y_true, y_pred, average='macro'))

5/5 [==============================] - 0s 2ms/step
[[859 141]
 [204 297]]
0.7326778366441129
